In [54]:
import random
import string
import re
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import MultiHeadAttention


In [7]:
# Завантаження даних
with open('ukr.txt', 'r', encoding='utf-8') as f:
    lines = f.read().strip().split('\n')

# Перевірка кількох випадкових рядків
for i in range(3):
    print(f"Приклад {i + 1}: {random.choice(lines)}")

Приклад 1: I doubt Tom could do that by himself.	Сумніваюся, що Том зміг би це зробити самостійно.	CC-BY 2.0 (France) Attribution: tatoeba.org #7186243 (CK) & #7879957 (deniko)
Приклад 2: I've been honest with you.	Я була чесна з тобою.	CC-BY 2.0 (France) Attribution: tatoeba.org #2359153 (CK) & #6969454 (deniko)
Приклад 3: I have a red car.	У мене червона машина.	CC-BY 2.0 (France) Attribution: tatoeba.org #3058028 (CK) & #5764541 (deniko)


In [29]:
text_pairs = []
for line in lines:
    eng, ukr, _ = line.split('\t')
    ukr = '[s] ' + ukr + ' [e]'
    text_pairs.append((eng, ukr))

for t in range(5):
    print(random.choice(text_pairs))

('I thought that was you.', '[s] Я думала, це була ти. [e]')
('Are you getting paid for doing this?', '[s] Тобі платять за те, що ти це робиш? [e]')
('We hope to arrive on time.', '[s] Ми сподіваємося приїхати вчасно. [e]')
('Dreaming costs nothing.', '[s] Сни безкоштовні. [e]')
('When will you take a bath?', '[s] Коли ти прийматимеш ванну? [e]')


In [8]:
# Перемішування і розбиття на набори
random.shuffle(text_pairs)
data_size = len(text_pairs)
num_val = int(0.15 * data_size)
num_train = data_size - 2 * num_val

train_pairs = text_pairs[:num_train]
val_pairs = text_pairs[num_train:num_train + num_val]
test_pairs = text_pairs[num_train + num_val:]

print(f"\nКількість пар: {data_size}")
print(f"Тренувальний набір: {len(train_pairs)}")
print(f"Валідаційний набір: {len(val_pairs)}")
print(f"Тестовий набір: {len(test_pairs)}")


Кількість пар: 50000
Тренувальний набір: 35000
Валідаційний набір: 7500
Тестовий набір: 7500


In [31]:
# Підготовка текстів
strip_chars = string.punctuation.replace('[', '').replace(']', '')
vocabulary_size = 15000
sequence_length = 20
batch_size = 64

# Функція стандартизації українських текстів
def standardize_ukr_text(text):
    return tf.strings.regex_replace(tf.strings.lower(text), f"[{re.escape(strip_chars)}]", '')

# Текстові векторизатори
eng_vector = keras.layers.TextVectorization(
    max_tokens=vocabulary_size,
    output_mode='int',
    output_sequence_length=sequence_length
)
ukr_vector = keras.layers.TextVectorization(
    max_tokens=vocabulary_size,
    output_mode='int',
    output_sequence_length=sequence_length + 1,
    standardize=standardize_ukr_text
)

train_eng_texts = [pair[0] for pair in train_pairs]
train_ukr_texts = [pair[1] for pair in train_pairs]

eng_vector.adapt(train_eng_texts)
ukr_vector.adapt(train_ukr_texts)

In [32]:
for v in [eng_vector, ukr_vector]:
    print(len(v.get_vocabulary()))
    print(v.get_vocabulary()[10:20])

6472
[np.str_('do'), np.str_('im'), np.str_('dont'), np.str_('he'), np.str_('mary'), np.str_('was'), np.str_('have'), np.str_('in'), np.str_('me'), np.str_('it')]
15000
[np.str_('на'), np.str_('Мері'), np.str_('ти'), np.str_('Тома'), np.str_('з'), np.str_('я'), np.str_('у'), np.str_('Це'), np.str_('У'), np.str_('в')]


In [33]:
def format_dataset(eng, ukr):
    eng = eng_vector(eng)
    ukr = ukr_vector(ukr)
    return ({"encoder_inputs": eng, "decoder_inputs": ukr[:, :-1]}, ukr[:, 1:])

def make_dataset(pairs):
    eng_texts, ukr_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    ukr_texts = list(ukr_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, ukr_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.cache().shuffle(2048).prefetch(16)

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [10]:
for inputs, targets in train_ds.take(1):
    print(f'encoder inputs shape: {inputs["encoder_inputs"].shape}')
    print(f'decoder inputs shape: {inputs["decoder_inputs"].shape}')
    print(f"targets shape: {targets.shape}")

encoder inputs shape: (64, 20)
decoder inputs shape: (64, 20)
targets shape: (64, 20)


In [66]:
# Генерація позиційного кодування
def get_positional_encoding(seq_len, embed_dim):
    position = tf.range(seq_len, dtype=tf.float32)[:, tf.newaxis]  # (seq_len, 1)
    div_term = tf.exp(tf.range(0, embed_dim, 2, dtype=tf.float32) * -(tf.math.log(10000.0) / embed_dim))  # (embed_dim // 2)
    
    # Compute sine and cosine for even and odd indices
    sinusoidal = tf.concat([tf.sin(position * div_term), tf.cos(position * div_term)], axis=-1)
    
    return sinusoidal

In [72]:
# Оновлення класів TransformerEncoder та TransformerDecoder з додаванням позиційного кодування
class TransformerEncoder(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, latent_dim):
        super(TransformerEncoder, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.latent_dim = latent_dim
        self.attention = MultiHeadAttention(num_heads, embed_dim)  # Assuming you have this implemented

    def call(self, inputs, training=False):
        seq_len = tf.shape(inputs)[1]
        pos_encoding = get_positional_encoding(seq_len, self.embed_dim)
        inputs = inputs + pos_encoding  # Add positional encoding to the input
        attn_output = self.attention(inputs, inputs)  # Example attention mechanism
        return attn_output

In [73]:
class TransformerDecoder(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerDecoder, self).__init__()
        self.attention1 = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.attention2 = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = tf.keras.Sequential([keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim)])
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)
        self.dropout3 = keras.layers.Dropout(rate)

    def call(self, inputs, enc_output, training=False):
        seq_len = tf.shape(inputs)[1]
        pos_encoding = get_positional_encoding(seq_len, inputs.shape[-1])
        inputs = inputs + pos_encoding  # Add positional encoding to the input
        attn_output1 = self.attention1(inputs, inputs)
        attn_output1 = self.dropout1(attn_output1, training=training)
        out1 = self.layernorm1(inputs + attn_output1)
        attn_output2 = self.attention2(out1, enc_output)
        attn_output2 = self.dropout2(attn_output2, training=training)
        out2 = self.layernorm2(out1 + attn_output2)
        ffn_output = self.dense_proj(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        return self.layernorm3(out2 + ffn_output)

In [74]:
embed_dim = 256
latent_dim = 2048
num_heads = 8

# Вхідні дані для енкодера
encoder_inputs = keras.layers.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = keras.layers.Embedding(input_dim=vocabulary_size, output_dim=embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, num_heads, latent_dim)(x, training=True)

# Вхідні дані для декодера
decoder_inputs = keras.layers.Input(shape=(None,), dtype="int64", name="decoder_inputs")
x = keras.layers.Embedding(input_dim=vocabulary_size, output_dim=embed_dim)(decoder_inputs)

# Підключення декодера до виходів енкодера
x = TransformerDecoder(embed_dim, num_heads, latent_dim)(x, encoder_outputs, training=True)
decoder_outputs = keras.layers.Dense(vocabulary_size, activation="softmax")(x)

# Створення моделі
transformer = keras.models.Model([encoder_inputs, decoder_inputs], decoder_outputs, name="transformer")



In [75]:
epochs = 5

transformer.compile("rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
transformer.fit(train_ds, epochs=epochs, validation_data=val_ds)

Epoch 1/5
547/547 ━━━━━━━━━━━━━━━━━━━━ 428s 772ms/step - accuracy: 0.7211 - loss: 2.5059 - val_accuracy: 0.7574 - val_loss: 1.6232
Epoch 2/5
547/547 ━━━━━━━━━━━━━━━━━━━━ 484s 885ms/step - accuracy: 0.7621 - loss: 1.6413 - val_accuracy: 0.7746 - val_loss: 1.4878
Epoch 3/5
547/547 ━━━━━━━━━━━━━━━━━━━━ 450s 822ms/step - accuracy: 0.7733 - loss: 1.5360 - val_accuracy: 0.7846 - val_loss: 1.3970
Epoch 4/5
547/547 ━━━━━━━━━━━━━━━━━━━━ 456s 833ms/step - accuracy: 0.7811 - loss: 1.4475 - val_accuracy: 0.7903 - val_loss: 1.3375
Epoch 5/5
547/547 ━━━━━━━━━━━━━━━━━━━━ 469s 858ms/step - accuracy: 0.7857 - loss: 1.3615 - val_accuracy: 0.7905 - val_loss: 1.2831


In [80]:
ukr_vocab = ukr_vector.get_vocabulary()
ukr_index_lookup = dict(zip(range(len(ukr_vocab)), ukr_vocab))
max_decoded_sentence_length = 20  

def decode_sequence(input_sentence):
    # Токенізація вхідного англійського речення
    tokenized_input_sentence = eng_vector([input_sentence])
    
    # Початковий токен для декодування
    decoded_sentence = "[s]" 
    
    # Генерація перекладу з кількох кроків
    for i in range(max_decoded_sentence_length):
        # Токенізація поточного перекладеного речення
        tokenized_target_sentence = ukr_vector([decoded_sentence])[:, :-1]
        
        # Прогнозування наступних токенів
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])
        
        # Вибір найбільш ймовірного токена на поточному кроці
        sampled_token_index = np.argmax(predictions[0, i, :])  
        sampled_token = ukr_index_lookup[sampled_token_index]  
        
        decoded_sentence += " " + sampled_token 
        
        # Якщо досягли кінцевого токена [e], припиняємо переклад
        if sampled_token == "[e]":
            break
    
    return decoded_sentence

# Тестування перекладу для кількох речень
test_eng_texts = [pair[0] for pair in test_pairs]

# Виведемо переклад для кількох речень
for _ in range(3):
    input_sentence = random.choice(test_eng_texts)  
    translated = decode_sequence(input_sentence)  
    print(f'--input: {input_sentence}')
    print(f'--output: {translated}')

--input: I've got a map.
--output: [s] Том ти не [e]
--input: Let's quit.
--output: [s] Том не [e]
--input: Do you know who made it?
--output: [s] Том ти не [e]
